In [ ]:

"""
IFCB ↔ PACE OCI L3M Rrs Matchup Deliverable Script
--------------------------------------------------
Creates a clean, reproducible deliverable for teammates:
- PACE L3M Rrs matchups (nearest pixel) for dates/locations in IFCB CSV
- Spectra plots per date/location
- Montage maps (clean land mask + visible black point)
- Feature tables (wide + long)
- IFCB daily labels aggregated into coarse phytoplankton groups
- Merged dataset (features + labels) suitable for ML training
- QC summary + README + requirements

INPUT
-----
IFCB CSV must include at least:
- date (YYYY-MM-DD or YYYYMMDD)
- latitude
- longitude

It should also include SOME taxonomic/class label column for grouping.
This script auto-detects from common names, or you can set IFCB_LABEL_COL env var.

PACE
----
Uses earthaccess to search and download OB.DAAC product:
short_name = PACE_OCI_L3M_RRS  (version varies server-side)

USAGE
-----
pip install -r requirements.txt
earthaccess login
python ifcb_pace_l3m_matchup_deliverable.py

Optional env vars:
- IFCB_CSV: path to ifcb.csv
- OUT_DIR: outputs directory
- PACE_SHORTNAME: default PACE_OCI_L3M_RRS
- TARGET_NM: montage wavelength (default 442)
- MONTAGE_HALF_WINDOW_DEG: default 1.0
- MONTAGE_NCOLS: default 4
- IFCB_LABEL_COL: force label column name
- IFCB_SITE_COL: optional site/station column name to include in grouping/merge
"""

from __future__ import annotations

import os
import re
import math
import json
import pathlib
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xarray as xr
import earthaccess

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib import cm


# -----------------------------
# Utilities
# -----------------------------

def ensure_dir(p: str | pathlib.Path) -> pathlib.Path:
    p = pathlib.Path(p)
    p.mkdir(parents=True, exist_ok=True)
    return p

def safe_fname(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9_.-]+", "_", s)

def parse_date_series(date_ser: pd.Series) -> pd.Series:
    s = date_ser.astype(str).str.strip()
    dt = pd.to_datetime(s, errors="coerce", format="%Y-%m-%d")
    m = dt.isna()
    if m.any():
        dt2 = pd.to_datetime(s[m], errors="coerce", format="%Y%m%d")
        dt.loc[m] = dt2
    if dt.isna().any():
        bad = s[dt.isna()].unique()[:10]
        raise ValueError(f"Could not parse some dates. Examples: {bad}")
    return dt.dt.normalize()

def haversine_km(lat1, lon1, lat2, lon2) -> float:
    R = 6371.0088
    phi1 = np.deg2rad(lat1)
    phi2 = np.deg2rad(lat2)
    dphi = np.deg2rad(lat2 - lat1)
    dl = np.deg2rad(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dl/2)**2
    return float(2 * R * np.arcsin(np.sqrt(a)))

def pick_first_existing_col(df: pd.DataFrame, candidates: list[str]) -> str | None:
    lower_map = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand.lower() in lower_map:
            return lower_map[cand.lower()]
    return None

def write_text(fp: pathlib.Path, text: str):
    fp.write_text(text, encoding="utf-8")


# -----------------------------
# IFCB label grouping
# -----------------------------

def default_group_mapping() -> dict[str, str]:
    """
    EDIT HERE if you want custom mapping.
    Keys are regex patterns matched against the IFCB label text (lowercased).
    Values are the coarse groups for the ML target.

    You can expand patterns as you learn your IFCB taxonomy strings.
    """
    return {
        r"\bdiatom": "diatoms",
        r"\bbacillari": "diatoms",
        r"\b(dino|dinoflag)": "dinoflagellates",
        r"\bhapto": "haptophytes",
        r"\bemiliania": "haptophytes",
        r"\bsynech|prochl|cyan": "cyanobacteria_like",
        r"\bcryptophy": "cryptophytes",
        # fallback handled separately
    }

def map_label_to_group(label: str, mapping: dict[str, str]) -> str:
    if label is None or (isinstance(label, float) and np.isnan(label)):
        return "other_unknown"
    s = str(label).strip().lower()
    if s == "" or s in {"nan", "none"}:
        return "other_unknown"
    for pat, grp in mapping.items():
        if re.search(pat, s):
            return grp
    return "other_unknown"

def aggregate_ifcb_labels(
    df: pd.DataFrame,
    date_col: str,
    lat_col: str,
    lon_col: str,
    label_col: str,
    site_col: str | None,
    out_fp: pathlib.Path,
) -> pd.DataFrame:
    mapping = default_group_mapping()
    tmp = df[[date_col, lat_col, lon_col, label_col] + ([site_col] if site_col else [])].copy()

    tmp["group"] = tmp[label_col].apply(lambda x: map_label_to_group(x, mapping))

    group_keys = [date_col, lat_col, lon_col]
    if site_col:
        group_keys.append(site_col)

    counts = tmp.groupby(group_keys + ["group"]).size().rename("n").reset_index()

    # total per day/location
    totals = counts.groupby(group_keys)["n"].sum().rename("n_total").reset_index()

    out = counts.merge(totals, on=group_keys, how="left")
    out["fraction"] = out["n"] / out["n_total"]

    # pivot wide fractions (nice for ML labels)
    frac_wide = out.pivot_table(
        index=group_keys,
        columns="group",
        values="fraction",
        fill_value=0.0,
        aggfunc="mean",
    ).reset_index()

    # also pivot counts wide
    count_wide = out.pivot_table(
        index=group_keys,
        columns="group",
        values="n",
        fill_value=0,
        aggfunc="sum",
    ).reset_index()
    count_wide.columns = [c if isinstance(c, str) else str(c) for c in count_wide.columns]
    frac_wide.columns = [c if isinstance(c, str) else str(c) for c in frac_wide.columns]

    # rename columns to make explicit
    for c in list(count_wide.columns):
        if c not in group_keys:
            count_wide = count_wide.rename(columns={c: f"count_{c}"})
    for c in list(frac_wide.columns):
        if c not in group_keys:
            frac_wide = frac_wide.rename(columns={c: f"frac_{c}"})

    labels = totals.merge(count_wide, on=group_keys, how="left").merge(frac_wide, on=group_keys, how="left")

    labels.to_csv(out_fp, index=False)
    return labels


# -----------------------------
# PACE access + matchup
# -----------------------------

def search_pace_l3m_rrs_for_date(date: datetime, short_name: str):
    t0 = date.strftime("%Y-%m-%d")
    t1 = (date + timedelta(days=1)).strftime("%Y-%m-%d")
    return earthaccess.search_data(short_name=short_name, temporal=(t0, t1), count=50)

def pick_best_granule(granules, prefer_resolution="4km"):
    if not granules:
        return None
    for g in granules:
        try:
            for u in g.data_links():
                if prefer_resolution.lower() in u.lower():
                    return g
        except Exception:
            pass
    return granules[0]

def open_rrs_dataset(granule) -> tuple[xr.Dataset, str]:
    paths = earthaccess.open([granule])
    if not paths:
        raise RuntimeError("earthaccess.open returned no paths.")
    fp = paths[0]
    # robust open
    for engine in (None, "h5netcdf", "netcdf4"):
        try:
            ds = xr.open_dataset(fp, engine=engine) if engine else xr.open_dataset(fp)
            return ds, str(fp)
        except Exception:
            continue
    raise RuntimeError(f"Could not open dataset: {fp}")

def nearest_rrs_spectrum(ds: xr.Dataset, lat: float, lon: float):
    lat_name = "lat" if "lat" in ds.coords else ("latitude" if "latitude" in ds.coords else None)
    lon_name = "lon" if "lon" in ds.coords else ("longitude" if "longitude" in ds.coords else None)
    if lat_name is None or lon_name is None:
        raise KeyError(f"Could not find lat/lon coords in dataset. Coords: {list(ds.coords)}")
    if "Rrs" not in ds.variables:
        raise KeyError(f"Could not find variable 'Rrs'. Variables: {list(ds.variables)}")
    if "wavelength" not in ds.coords and "wavelength" not in ds.variables:
        raise KeyError("Could not find 'wavelength' coordinate/variable.")

    # handle 0..360 lon in file
    lons = ds[lon_name].values
    lon_use = lon + 360.0 if (np.nanmin(lons) >= 0 and lon < 0) else lon

    rrs = ds["Rrs"]
    wl = ds["wavelength"].values

    sel = rrs.sel({lat_name: lat, lon_name: lon_use}, method="nearest")
    lat_near = float(sel[lat_name].values)
    lon_near = float(sel[lon_name].values)
    lon_near_out = lon_near - 360.0 if lon_near > 180 else lon_near
    spec = sel.values

    return wl, spec, lat_near, lon_near_out

def rrs_at_wavelength(ds: xr.Dataset, lat: float, lon: float, target_nm: float):
    wl, spec, lat_near, lon_near = nearest_rrs_spectrum(ds, lat, lon)
    idx = int(np.nanargmin(np.abs(wl - target_nm)))
    return float(wl[idx]), float(spec[idx]), lat_near, lon_near


# -----------------------------
# Plotting
# -----------------------------

def plot_spectrum(wl, spec, title: str, out_png: pathlib.Path):
    plt.figure(figsize=(8.5, 4.8))
    plt.plot(wl, spec)
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("Rrs (sr$^{-1}$)")
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(out_png, dpi=220)
    plt.close()

def plot_montage_maps_clean(
    ds_list,
    meta_list,
    *,
    target_nm: float,
    out_png: pathlib.Path,
    half_window_deg: float = 1.0,
    ncols: int = 4,
):
    import numpy as np
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    from matplotlib import cm

    def _subset_rrs(ds, lat0, lon0, wli, half_window_deg):
        lon_name = "lon" if "lon" in ds.coords else "longitude"
        lat_name = "lat" if "lat" in ds.coords else "latitude"

        # selection lon handling (0..360)
        lons = ds[lon_name].values
        lon_use = lon0 + 360.0 if (np.nanmin(lons) >= 0 and lon0 < 0) else lon0

        lat_min, lat_max = lat0 - half_window_deg, lat0 + half_window_deg
        lon_min, lon_max = lon_use - half_window_deg, lon_use + half_window_deg

        rrs2d = ds["Rrs"].isel(wavelength=wli)

        latv = ds[lat_name]
        lonv = ds[lon_name]
        mask = (latv >= lat_min) & (latv <= lat_max) & (lonv >= lon_min) & (lonv <= lon_max)
        sub = rrs2d.where(mask, drop=True)

        # if empty
        if (sub.sizes.get(lat_name, 0) < 2) or (sub.sizes.get(lon_name, 0) < 2):
            return None, None, None, None, lon_name, lat_name, lon_use

        # ONLY NOW load small window into memory
        sub_vals = sub.values.astype(float)

        fill = sub.attrs.get("_FillValue", None)
        if fill is not None:
            sub_vals = np.where(sub_vals == float(fill), np.nan, sub_vals)

        # mask invalid/land-ish
        sub_vals = np.where(sub_vals <= 0, np.nan, sub_vals)

        lon_plot = sub[lon_name].values.astype(float)
        lat_plot = sub[lat_name].values.astype(float)

        # wrap lon to [-180,180) for plotting
        if np.nanmax(lon_plot) > 180:
            lon_plot = ((lon_plot + 180) % 360) - 180
            order = np.argsort(lon_plot)
            lon_plot = lon_plot[order]
            sub_vals = sub_vals[:, order]

        return lon_plot, lat_plot, sub_vals, sub, lon_name, lat_name, lon_use

    n = len(ds_list)
    if n == 0:
        return

    # wavelength indices (cheap)
    wlis = []
    wl_used_list = []
    for ds in ds_list:
        wl = ds["wavelength"].values
        wli = int(np.nanargmin(np.abs(wl - target_nm)))
        wlis.append(wli)
        wl_used_list.append(float(wl[wli]))

    # --- shared color scale using ONLY small windows ---
    window_values = []
    for ds, meta, wli in zip(ds_list, meta_list, wlis):
        lat0 = float(meta["lat"])
        lon0 = float(meta["lon"])
        lon_plot, lat_plot, sub_vals, _, *_ = _subset_rrs(ds, lat0, lon0, wli, half_window_deg)
        if sub_vals is None:
            continue
        vv = sub_vals[np.isfinite(sub_vals)]
        if vv.size:
            window_values.append(vv)

    if window_values:
        allv = np.concatenate(window_values)
        vmin = float(np.nanpercentile(allv, 5))
        vmax = float(np.nanpercentile(allv, 98))
        if not np.isfinite(vmin) or not np.isfinite(vmax) or vmax <= vmin:
            vmin, vmax = float(np.nanmin(allv)), float(np.nanmax(allv))
    else:
        vmin, vmax = 0.0, 1.0

    cmap = cm.get_cmap("viridis").copy()
    cmap.set_bad(color="white", alpha=1.0)

    ncols = max(1, ncols)
    nrows = int(np.ceil(n / ncols))
    fig = plt.figure(figsize=(4.2 * ncols, 3.6 * nrows))
    last_im = None

    for i, (ds, meta, wli, wl_used) in enumerate(zip(ds_list, meta_list, wlis, wl_used_list), start=1):
        ax = plt.subplot(nrows, ncols, i, projection=ccrs.PlateCarree())

        lat0 = float(meta["lat"])
        lon0 = float(meta["lon"])

        lon_plot, lat_plot, sub_vals, _, *_ = _subset_rrs(ds, lat0, lon0, wli, half_window_deg)

        if sub_vals is None:
            ax.set_title(f"{meta['date_str']}\n(empty window)", fontsize=10)
            ax.add_feature(cfeature.LAND, facecolor="white", zorder=5)
            ax.add_feature(cfeature.COASTLINE.with_scale("50m"), linewidth=0.7, zorder=6)
            ax.plot(lon0, lat0, "o", ms=7, color="black", markeredgecolor="white",
                    markeredgewidth=1.8, transform=ccrs.PlateCarree(), zorder=10)
            continue

        last_im = ax.pcolormesh(
            lon_plot,
            lat_plot,
            sub_vals,
            transform=ccrs.PlateCarree(),
            shading="nearest",
            cmap=cmap,
            vmin=vmin,
            vmax=vmax,
            rasterized=True,
            zorder=1,
        )

        # clean coast/land
        ax.add_feature(cfeature.LAND, facecolor="white", zorder=5)
        ax.add_feature(cfeature.COASTLINE.with_scale("50m"), linewidth=0.7, zorder=6)

        # visible point
        ax.plot(
            lon0, lat0,
            marker="o",
            markersize=9,
            color="black",
            markeredgecolor="white",
            markeredgewidth=1.6,
            transform=ccrs.PlateCarree(),
            zorder=10,
        )

        ax.set_extent([float(np.min(lon_plot)), float(np.max(lon_plot)),
                       float(np.min(lat_plot)), float(np.max(lat_plot))],
                      crs=ccrs.PlateCarree())

        ax.set_title(f"{meta['date_str']}\nRrs ~ {wl_used:.0f} nm", fontsize=10)

    if last_im is not None:
        cax = fig.add_axes([0.92, 0.15, 0.015, 0.7])
        cb = plt.colorbar(last_im, cax=cax)
        cb.set_label("Rrs (sr$^{-1}$)")

    plt.tight_layout(rect=[0, 0, 0.9, 1])
    plt.savefig(out_png, dpi=240)
    plt.close(fig)



# -----------------------------
# Main pipeline
# -----------------------------

def main(
    ifcb_csv: str,
    out_dir: str,
    pace_short_name: str,
    target_nm: float,
    montage_half_window_deg: float,
    montage_ncols: int,
):
    outp = ensure_dir(out_dir)
    log_fp = outp / "log.txt"

    def log(msg: str):
        ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        line = f"[{ts}] {msg}"
        print(line)
        with open(log_fp, "a", encoding="utf-8") as f:
            f.write(line + "\n")

    # Load IFCB
    df = pd.read_csv(ifcb_csv)
    # auto-detect required columns
    date_col = pick_first_existing_col(df, ["date", "Date", "sample_date"])
    lat_col  = pick_first_existing_col(df, ["latitude", "lat", "Latitude", "LAT"])
    lon_col  = pick_first_existing_col(df, ["longitude", "lon", "Longitude", "LON", "long"])
    if not (date_col and lat_col and lon_col):
        raise KeyError(f"Need date/lat/lon columns. Found: {list(df.columns)}")

    df[date_col] = parse_date_series(df[date_col])
    df[lat_col] = pd.to_numeric(df[lat_col], errors="coerce")
    df[lon_col] = pd.to_numeric(df[lon_col], errors="coerce")

    # optional site/station column
    site_col = os.environ.get("IFCB_SITE_COL", "").strip() or None
    if site_col and site_col not in df.columns:
        log(f"WARNING: IFCB_SITE_COL={site_col} not found; ignoring.")
        site_col = None
    if not site_col:
        site_col = pick_first_existing_col(df, ["site", "station", "Station", "site_id", "station_id"])

    # label column detection
    forced_label = os.environ.get("IFCB_LABEL_COL", "").strip()
    if forced_label:
        if forced_label not in df.columns:
            raise KeyError(f"IFCB_LABEL_COL={forced_label} not in columns.")
        label_col = forced_label
    else:
        label_col = pick_first_existing_col(df, [
            "autoclass", "class", "taxon", "taxonomy", "label", "species", "scientific_name",
            "classification", "annotated_class", "predicted_class"
        ])
    if not label_col:
        log("WARNING: No label column detected. Labels file will NOT be created. "
            "Set IFCB_LABEL_COL to your column name if you have it.")
    else:
        log(f"Detected IFCB label column: {label_col}")

    # Unique points (date + lat/lon [+ site])
    keys = [date_col, lat_col, lon_col] + ([site_col] if site_col else [])
    pts = df[keys].dropna().drop_duplicates().sort_values(date_col)

    log(f"Loaded {len(df):,} IFCB rows, unique date/lat/lon combos: {len(pts)}")

    # Earthdata login
    log("Logging into Earthdata (earthaccess)...")
    earthaccess.login()

    # storage
    feature_rows = []
    long_rows = []
    ds_for_montage = []
    meta_for_montage = []
    ok_count, fail_count = 0, 0

    # iterate unique date/location combos
    for _, r in pts.iterrows():
        date = pd.Timestamp(r[date_col]).to_pydatetime()
        date_str = date.strftime("%Y-%m-%d")
        lat = float(r[lat_col])
        lon = float(r[lon_col])
        site_val = str(r[site_col]) if site_col else None

        log(f"Searching PACE L3M Rrs for {date_str} ...")
        granules = search_pace_l3m_rrs_for_date(date, short_name=pace_short_name)
        if not granules:
            fail_count += 1
            feature_rows.append({
                "date": date_str, "latitude": lat, "longitude": lon,
                "site": site_val,
                "status": "no_granule",
            })
            continue

        g = pick_best_granule(granules, prefer_resolution="4km")
        if g is None:
            fail_count += 1
            feature_rows.append({
                "date": date_str, "latitude": lat, "longitude": lon,
                "site": site_val,
                "status": "no_selection",
            })
            continue

        try:
            ds, local_fp = open_rrs_dataset(g)
        except Exception as e:
            fail_count += 1
            feature_rows.append({
                "date": date_str, "latitude": lat, "longitude": lon,
                "site": site_val,
                "status": "open_failed",
                "error": str(e),
            })
            continue

        try:
            wl, spec, lat_near, lon_near = nearest_rrs_spectrum(ds, lat, lon)
        except Exception as e:
            fail_count += 1
            feature_rows.append({
                "date": date_str, "latitude": lat, "longitude": lon,
                "site": site_val,
                "status": "extract_failed",
                "error": str(e),
            })
            try:
                ds.close()
            except Exception:
                pass
            continue

        ok_count += 1

        # distance to nearest pixel
        dist_km = haversine_km(lat, lon, lat_near, lon_near)

        # Save spectra plot
        spec_png = outp / safe_fname(f"spectra_{date_str}_lat{lat:.4f}_lon{lon:.4f}.png")
        plot_spectrum(
            wl,
            spec,
            title=f"PACE OCI L3M Rrs spectrum\n{date_str} @ ({lat:.4f}, {lon:.4f}) "
                  f"nearest=({lat_near:.4f}, {lon_near:.4f}) dist={dist_km:.2f} km",
            out_png=spec_png,
        )
        log(f"Saved spectra: {spec_png.name}")

        # Save montage inputs
        ds_for_montage.append(ds)
        meta_for_montage.append({
            "date_str": date_str,
            "lat": lat,
            "lon": lon,
        })

        # row wide features
        row = {
            "date": date_str,
            "latitude": lat,
            "longitude": lon,
            "nearest_lat": lat_near,
            "nearest_lon": lon_near,
            "distance_km": dist_km,
            "pace_short_name": pace_short_name,
            "status": "ok",
            "pace_local_file": local_fp,
        }
        if site_col:
            row["site"] = site_val

        # target wavelength
        w_used, r_target, _, _ = rrs_at_wavelength(ds, lat, lon, target_nm)
        row["target_nm_requested"] = float(target_nm)
        row["target_nm_used"] = float(w_used)
        row["rrs_target"] = float(r_target) if np.isfinite(r_target) else np.nan

        # add all wavelengths as rrs_<nm>
        for w, v in zip(wl, spec):
            w_int = int(round(float(w)))
            row[f"rrs_{w_int}"] = float(v) if np.isfinite(v) else np.nan

            long_rows.append({
                "date": date_str,
                "latitude": lat,
                "longitude": lon,
                "site": site_val,
                "wavelength_nm": float(w),
                "rrs": float(v) if np.isfinite(v) else np.nan,
                "nearest_lat": lat_near,
                "nearest_lon": lon_near,
                "distance_km": dist_km,
                "status": "ok",
            })

        feature_rows.append(row)

    # Write feature tables
    features_wide = pd.DataFrame(feature_rows)
    out_wide = outp / "matchups_features_wide.csv"
    features_wide.to_csv(out_wide, index=False)
    log(f"Wrote: {out_wide}")

    features_long = pd.DataFrame(long_rows)
    out_long = outp / "matchups_features_long.csv"
    features_long.to_csv(out_long, index=False)
    log(f"Wrote: {out_long}")

    # Montage map (clean)
    if ds_for_montage:
        montage_png = outp / safe_fname(f"montage_rrs_{int(round(target_nm))}nm.png")
        plot_montage_maps_clean(
            ds_for_montage,
            meta_for_montage,
            target_nm=target_nm,
            out_png=montage_png,
            half_window_deg=montage_half_window_deg,
            ncols=montage_ncols,
        )
        log(f"Wrote: {montage_png}")

    # Close datasets
    for ds in ds_for_montage:
        try:
            ds.close()
        except Exception:
            pass

    # Labels aggregation (if possible)
    labels_fp = outp / "ifcb_labels_daily.csv"
    labels = None
    if label_col:
        labels = aggregate_ifcb_labels(
            df=df,
            date_col=date_col,
            lat_col=lat_col,
            lon_col=lon_col,
            label_col=label_col,
            site_col=site_col,
            out_fp=labels_fp,
        )
        log(f"Wrote: {labels_fp}")

    # Merge features + labels (deliverable for ML)
    merged_fp = outp / "matchups_with_labels.csv"
    if labels is not None:
        # align column names for merge
        f = features_wide.copy()
        f["date"] = pd.to_datetime(f["date"])
        lbl = labels.copy()
        lbl = lbl.rename(columns={
            date_col: "date",
            lat_col: "latitude",
            lon_col: "longitude",
        })
        lbl["date"] = pd.to_datetime(lbl["date"])

        merge_keys = ["date", "latitude", "longitude"]
        if site_col and "site" in f.columns and "site" in lbl.columns:
            merge_keys.append("site")

        merged = f.merge(lbl, on=merge_keys, how="left")
        merged["date"] = merged["date"].dt.strftime("%Y-%m-%d")
        merged.to_csv(merged_fp, index=False)
        log(f"Wrote: {merged_fp}")
    else:
        # still write a copy for consistency
        features_wide.to_csv(merged_fp, index=False)
        log(f"Wrote (no labels available): {merged_fp}")

    # QC summary
    qc = {
        "n_ifcb_rows": int(len(df)),
        "n_unique_date_lat_lon": int(len(pts)),
        "n_matchup_ok": int(ok_count),
        "n_matchup_failed": int(fail_count),
        "pace_product_short_name": pace_short_name,
        "target_nm_for_montage": float(target_nm),
        "montage_half_window_deg": float(montage_half_window_deg),
    }
    qc_df = pd.DataFrame([qc])
    qc_fp = outp / "qc_summary.csv"
    qc_df.to_csv(qc_fp, index=False)
    log(f"Wrote: {qc_fp}")

    # README deliverable
    readme = f"""# IFCB ↔ PACE OCI L3M Rrs Matchup Deliverable

## What this deliverable contains
- **matchups_features_wide.csv**: ML-ready features table (Rrs at many wavelengths as columns).
- **matchups_features_long.csv**: tidy long-format table (one row per wavelength).
- **ifcb_labels_daily.csv**: daily IFCB labels aggregated into coarse phytoplankton groups (counts + fractions).
- **matchups_with_labels.csv**: merged features + labels for model training.
- **montage_rrs_{int(round(target_nm))}nm.png**: daily montage map around the IFCB location.
- **spectra_*.png**: Rrs spectrum plot per date/location.
- **qc_summary.csv**: summary stats + configuration.
- **log.txt**: run log.

## PACE source
- Product short_name used: **{pace_short_name}**
- Matchup method: nearest-neighbor pixel in the daily L3M mapped product.

## Important note on spatial resolution
This dataset uses **L3M**. Typical L3M resolutions may be ~4 km (product-dependent).  
If the downstream goal is ~1 km coastal community maps, consider switching to **L2 swath** or a finer-resolution gridded product.

## Label definition
IFCB class/taxon labels were mapped into coarse groups via regex rules in:
`default_group_mapping()` inside the script.

## Reproducibility
- Requires Earthdata login (`earthaccess login`)
- Dependencies pinned in `requirements.txt`
"""
    write_text(outp / "README_deliverable.md", readme)
    log("Wrote: README_deliverable.md")

    # requirements
    req = """earthaccess
xarray
netcdf4
h5netcdf
pandas
numpy
matplotlib
cartopy
"""
    write_text(outp / "requirements.txt", req)
    log("Wrote: requirements.txt")

    log("DONE.")


if __name__ == "__main__":
    IFCB_CSV = os.environ.get("IFCB_CSV", "pace-flow/DATA_TEST/ifcb.csv")
    OUT_DIR = os.environ.get("OUT_DIR", "outputs")
    PACE_SHORTNAME = os.environ.get("PACE_SHORTNAME", "PACE_OCI_L3M_RRS")

    TARGET_NM = float(os.environ.get("TARGET_NM", "442"))
    MONTAGE_HALF_WINDOW_DEG = float(os.environ.get("MONTAGE_HALF_WINDOW_DEG", "1.0"))
    MONTAGE_NCOLS = int(os.environ.get("MONTAGE_NCOLS", "4"))

    main(
        ifcb_csv=IFCB_CSV,
        out_dir=OUT_DIR,
        pace_short_name=PACE_SHORTNAME,
        target_nm=TARGET_NM,
        montage_half_window_deg=MONTAGE_HALF_WINDOW_DEG,
        montage_ncols=MONTAGE_NCOLS,
    )
